In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import time

from pygame import mixer

In [ ]:
l=[]
mixer.init()
mixer.music.load('beep.mp3')
for i in range(80):
    wd = webdriver.Firefox()
    wd.get('http://www.upsldc.org/real-time-data')
    #wait till dyamic loaded content show up
    WebDriverWait(wd, 50).until(
        EC.visibility_of_element_located((By.CLASS_NAME, "up_schedule")))

    html_page = wd.page_source
    wd.quit()

    soup = BeautifulSoup(html_page)

    d={}
    all=soup.find_all("td")
    d['time']=dt.datetime.now()
    d['Schedule (MW)']=all[1].get_text()
    d['Drawl (MW)']=all[3].get_text()
    d['Demand (MW)']=all[7].get_text()
    if(d['Demand (MW)']>'14000'):
        mixer.music.play()
    d['Total SSGS (MW)']=all[9].get_text()
    d['UP Thermal Generation (MW)']=all[11].get_text()
    d['IPP Thermal Generation (MW)']=all[13].get_text()
    d['UP Hydro Generation (MW)']=all[15].get_text()
    d['Freq (Hz)']=all[21].get_text()
    d['OD/UD (MW)']=all[5].get_text()
    l.append(d)
    
    time.sleep(30)

df=pd.DataFrame(l)


In [ ]:
df=to_csv("hfs.csv")

In [1]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.io import output_notebook
import pandas as pd

def dsd_curve():
    df = pd.read_csv('up12_2_34pmf.csv') 
    df['time'] = pd.to_datetime(df['time'])
    df["time_format"]=df["time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    cds=ColumnDataSource(df)
    p = figure(width=1000, height=500, x_axis_type='datetime', title="dsd vs time")

    p1=p.line(x='time', y='Demand (MW)', color='red', source=cds)
    p.add_tools(HoverTool(renderers=[p1], tooltips=[('Name','Demand curve'),('time',"@time_format")]))
    p.circle(x='time', y='Demand (MW)', color='blue',size=3, alpha=0.5, source=cds)

    p2=p.line(x='time', y='Schedule (MW)', color='indigo', source=cds)
    p.add_tools(HoverTool(renderers=[p2], tooltips=[('Name','Schedule curve'),('time',"@time_format")]))
    p.circle(x='time', y='Schedule (MW)', color='red',size=3,alpha=0.5, source=cds)

    p3=p.line(x='time', y='Drawl (MW)', color='teal', source=cds)
    p.add_tools(HoverTool(renderers=[p3], tooltips=[('Name','Drawl curve'),('time',"@time_format")]))
    p.circle(x='time', y='Drawl (MW)', color='purple',size=3, source=cds)

    p.xaxis.axis_label='timestamp'
    p.yaxis.axis_label='Demand, Schedule and Drawl (MW)'
    p.xaxis.formatter=DatetimeTickFormatter(
        seconds=["%H:%M:%S"],
        minsec=["%H:%M:%S"],
        minutes=["%H:%M:%S"],
        hourmin=["%H:%M:%S"],
        hours=["%H:%M:%S"],
        )
    output_file("ds.html")
    output_notebook()
    show(p)
    
def gen_curve():
    df = pd.read_csv('up12_2_34pmf.csv') 
    df['time'] = pd.to_datetime(df['time'])
    df["time_format"]=df["time"].dt.strftime("%Y-%m-%d %H:%M:%S")
    cds=ColumnDataSource(df)

    p = figure(width=1000, height=500, x_axis_type='datetime', title="power generation vs time")

    p1=p.line(x='time', y='Total SSGS (MW)', color='red', source=cds)
    p.add_tools(HoverTool(renderers=[p1], tooltips=[('Name','Total SSGS (MW)'),('time',"@time_format")]))
    p.circle(x='time', y='Total SSGS (MW)', color='blue',size=3, alpha=0.5, source=cds)

    p2=p.line(x='time', y='IPP Thermal Generation (MW)', color='indigo', source=cds)
    p.add_tools(HoverTool(renderers=[p2], tooltips=[('Name','IPP Thermal Generation (MW)'),('time',"@time_format")]))
    p.circle(x='time', y='IPP Thermal Generation (MW)', color='red',size=3, alpha=0.5, source=cds)

    p3=p.line(x='time', y='UP Thermal Generation (MW)', color='teal', source=cds)
    p.add_tools(HoverTool(renderers=[p3], tooltips=[('Name','UP Thermal Generation (MW)'),('time',"@time_format")]))
    p.circle(x='time', y='UP Thermal Generation (MW)', color='purple',size=3, alpha=0.5, source=cds)

    p.xaxis.axis_label='time'
    p.yaxis.axis_label='Power generation (MW)'
    p.xaxis.formatter=DatetimeTickFormatter(
        seconds=["%H:%M:%S"],
        minsec=["%H:%M:%S"],
        minutes=["%H:%M:%S"],
        hourmin=["%H:%M:%S"],
        hours=["%H:%M:%S"],
        )
    output_file("gen.html")
    output_notebook()
    show(p)
    
def odud_curve():
    dfo = pd.read_csv('up12_2_34pmf.csv')
    dfo['time'] = pd.to_datetime(dfo['time'])
    p = figure(plot_width=1000, plot_height=500, x_axis_type="datetime", title="surplus(ud)(-) / shortage(od)(+) vs time")

    p.line(dfo['time'], dfo['OD/UD (MW)'], color='navy', alpha=0.5)
    p.circle(dfo['time'], dfo['OD/UD (MW)'], color='red', size=4, alpha=0.5)

    p.xaxis.axis_label='time'
    p.yaxis.axis_label='Surplus/shortage (MW)'
    p.xaxis.formatter=DatetimeTickFormatter(
        seconds=["%H:%M:%S"],
        minsec=["%H:%M:%S"],
        minutes=["%H:%M:%S"],
        hourmin=["%H:%M:%S"],
        hours=["%H:%M:%S"],
        )
    output_file("odud.html")
    output_notebook()
    show(p)
    
def freq_curve():
    dff = pd.read_csv('up12_2_34pmf.csv')
    dff['time'] = pd.to_datetime(dff['time'])
    p = figure(plot_width=1000, plot_height=500, x_axis_type="datetime", title="freq vs time")

    p.line(dff['time'], dff['Freq (Hz)'], color='navy', alpha=0.5)
    p.circle(dff['time'], dff['Freq (Hz)'], color='red', size=4, alpha=0.5)

    p.xaxis.axis_label='time'
    p.yaxis.axis_label='Frequency (Hz)'
    p.xaxis.formatter=DatetimeTickFormatter(
        seconds=["%H:%M:%S"],
        minsec=["%H:%M:%S"],
        minutes=["%H:%M:%S"],
        hourmin=["%H:%M:%S"],
        hours=["%H:%M:%S"],
        )
    output_file("freq.html")
    show(p)
    
   
x=input("""select graph
1. demand,schedule,drawl
2. Power generation 
3. Surplus/ shortage
4. Frequency vs time : """)

if x=='1':
    dsd_curve()
elif x=='2':
    gen_curve()
elif x=='3':
    odud_curve()
else:
    freq_curve()

select graph
1. demand,schedule,drawl
2. Power generation 
3. Surplus/ shortage
4. Frequency vs time : 1


Loading BokehJS ...